In [522]:
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import random

@dataclass
class Layer:
    pass

@dataclass
class Input(Layer):
    out_features: int

@dataclass
class Dense(Layer):
    out_features: int
    learning_rate: float

@dataclass
class Conv(Layer):
    kernel_size: tuple[int,int]
    learning_rate: float

@dataclass
class Attention(Layer):
    learning_rate: float

@dataclass
class BatchNorm(Layer):
    pass

@dataclass
class SkipConn(Layer):
    from_offset: int
    learning_rate: float

In [527]:
model = [
    Input(2),
    Dense(2, .3)
]

In [ ]:
def generate_torch_model(layers: list[Layer]):
    

In [500]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import random

class Layer():
    def __init__(self, in_features, out_features, learning_rate):
        self.layer = nn.Linear(in_features, out_features)
        self.actication = nn.functional.relu

        self.learning_rate = learning_rate

    def update(self):
        with torch.no_grad():
            self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
            self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

    def __call__(self, x):
        return self.actication(self.layer(x))

In [ ]:
[Input(4), Dense(2), Skip(-2), Dense(6), Attention(2)]

In [504]:
class Layer():
    def __init__(self, prior, out_features):
        self.prior = prior
        self.out_features = out_features

    def update(self):
        raise NotImplementedError()

    def __call__(self, x):
        return self.prior(x)

    def iter(self, ret=None):
        if ret is None:
            ret = [self]
        return self.prior.iter() + ret


class Input(Layer):
    def __init__(self, out_features):
        super().__init__(None, out_features)

    def update(self):
        pass

    def __call__(self, x):
        return x

    def iter(self):
        return [self]
    
class NumOpLayer(Layer):
    def __init__(self, prior, out_features, learning_rate=None):
        super().__init__(prior, out_features)
        if learning_rate is None:
            learning_rate = random()
        self.layer = nn.Linear(self.prior.out_features, out_features)
        self.actication = F.relu
        self.learning_rate = learning_rate

    def update(self):
        with torch.no_grad():
            self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
            self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

    def __call__(self, x):
        return self.actication(self.layer(self.prior(x)))

class Dense(Layer):
    def __init__(self, prior, out_features, learning_rate=None):
        super().__init__(prior, out_features)
        if learning_rate is None:
            learning_rate = random()
        self.layer = nn.Linear(self.prior.out_features, out_features)
        self.actication = F.relu
        self.learning_rate = learning_rate

    def update(self):
        with torch.no_grad():
            self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
            self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

    def __call__(self, x):
        return self.actication(self.layer(self.prior(x)))

class Dense(Layer):
    def __init__(self, prior, out_features, learning_rate=None):
        super().__init__(prior, out_features)
        if learning_rate is None:
            learning_rate = random()
        self.layer = nn.Attention(prior.out_features, 1)
        self.actication = F.relu
        self.learning_rate = learning_rate

    def update(self):
        with torch.no_grad():
            self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
            self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

    def __call__(self, x):
        return self.actication(self.layer(self.prior(x)))

# class Layer():
#     def __init__(self, prior, out_features, learning_rate):
#         self.layer = nn.Linear(in_features, out_features)
#         self.actication = nn.functional.relu

#         self.prior = prior
#         self.learning_rate = learning_rate
#         self.out_features = out_features

#     def update(self):
#         with torch.no_grad():
#             self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
#             self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

#     def __call__(self, x):
#         return self.actication(self.layer(x))

In [634]:
attn = nn.MultiheadAttention(4, 1, batch_first=True)
conv = nn.Conv1d(3, 1, 2)
# conv2 = nn.Conv2d(2, 1, 2)
lin = nn.Linear(3,4)

In [635]:
x = lin(torch.Tensor([
    # [
    # [0, 1, 0],
    # [1, 2, 0]
    # ],
    # [
    [1, 0, 1],
    [1, 2, 1],
    [1, 2, 1]
    # ]
]))
print(x,'\n')
x, _ = attn(x, x, x, need_weights=False)
x
print(x, '\n')
conv(x)

tensor([[-0.6899, -0.3772,  0.4068, -0.5882],
        [ 0.3314, -0.2893,  0.2962,  0.3016],
        [ 0.3314, -0.2893,  0.2962,  0.3016]], grad_fn=<AddmmBackward0>) 

tensor([[-0.0205, -0.0167, -0.0051,  0.0785],
        [-0.0085, -0.0100, -0.0147,  0.0618],
        [-0.0085, -0.0100, -0.0147,  0.0618]], grad_fn=<SqueezeBackward1>) 



tensor([[-0.2044, -0.2029, -0.2248]], grad_fn=<SqueezeBackward1>)

In [637]:
class AbstractLayer:
    def __init__(self, prior:AbstractLayer, out_features:int, learning_rate:float):
        self.prior = propr
        self.out_features = out_features
        self.learning_rate = learning_rate

    def update(self):
        raise NotImplementedError()

    def __call__(self, x):
        return prior(x)

class Input(AbstractLayer):
    def __init__(self, out_features:int):
        super().__init__(out_features, 0)

    def __call__(self, x):
        return x

class Dense(AbstractLayer):
    def __init__(self, prior:AbstractLayer, out_features:int, learning_rate:float):
        super().__init__(prior, out_features, learning_rate)
        self.layer = nn.Linear(prior.out_features, out_features)

    def update(self):
        with torch.no_grad():
            self.layer.bias += self.learning_rate * torch.randn_like(self.layer.bias)
            self.layer.weight += self.learning_rate * torch.randn_like(self.layer.weight)

    def __call__(self, x):
        return self.layer(super().__call__(x))

class Conv(AbstractLayer):
    def __init__(self, prior:AbstractLayer, kernel_size:int, channels:int, learning_rate:float):
        
        self.layer = nn.Conv1d()
        

In [641]:
conv.kernel_size

(2,)

In [646]:
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
    return h, w

conv_output_shape((2,4), conv.kernel_size[0])

(1, 3)

In [ ]:
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
    return h, w

In [639]:
conv.kernel_size

(2,)

In [512]:
a = torch.Tensor([1,2,3]).unsqueeze(0).unsqueeze(0)
nn.MultiheadAttention(3, 1)(a,a,a)

(tensor([[[-1.1052,  0.2616, -0.0619]]], grad_fn=<ViewBackward0>),
 tensor([[[1.]]], grad_fn=<MeanBackward1>))

In [498]:
x = Input(2)
x = Dense(x, 3)
x = Dense(x, 4)

In [499]:
list(x.iter())

In [468]:
x(torch.Tensor([1,2]))

tensor([0.0000, 0.0547, 0.3688, 0.0000], grad_fn=<ReluBackward0>)

In [419]:
l = Layer(1,2,0.1)

In [438]:
l.update()

In [439]:
l(torch.Tensor([5]))

tensor([6.4307, 4.3782], grad_fn=<ReluBackward0>)

In [9]:
import torch
import torch.nn as nn

In [52]:
dense = nn.Linear(1,2,bias=True)

In [144]:
x = torch.Tensor([0])
dense(x)

tensor([3.1076, 3.0439], grad_fn=<ViewBackward0>)

In [143]:
with torch.no_grad():
    dense.bias += torch.randn_like(dense.bias)
    dense.weights += torch.randn_like(dense.weights)

In [50]:
torch.nn.functional.relu(torch.Tensor([-2]))

tensor([0.])

In [46]:
list(dense.parameters())

[Parameter containing:
 tensor([[0.9067]], requires_grad=True),
 Parameter containing:
 tensor([-0.6769], requires_grad=True)]